In [38]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [39]:
AGENT_NAME = "at030-custom-activities"

In [40]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='0927b469-5aed-4804-aa53-79a6af70f76f', created_at=datetime.datetime(2025, 8, 14, 5, 54, 44, 734709, tzinfo=TzInfo(UTC)), name='26d1fa25-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 8, 14, 5, 54, 44, 734709, tzinfo=TzInfo(UTC)))


In [41]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

num_events = 30
for i in range(num_events):
    rpc_response = client.agents.send_event(
        agent_name=AGENT_NAME,
        params={
            "content": {"type": "text", "author": "user", "content": f"Hello what can you do? EVENT NUM: {i}"},
            "task_id": task.id,
        }
    )
    
    event = rpc_response.result
    print(event)

Event(id='5f402c77-ed37-4f56-b161-50f3ceb87685', agent_id='c9fc2e91-df7a-42b4-bc79-fe154bd4db5a', sequence_id=247, task_id='0927b469-5aed-4804-aa53-79a6af70f76f', content=TextContent(author='user', content='Hello what can you do? EVENT NUM: 0', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 8, 14, 5, 54, 52, 106969, tzinfo=TzInfo(UTC)))
Event(id='f71c4b80-6d93-4167-bdf9-d2f407bde759', agent_id='c9fc2e91-df7a-42b4-bc79-fe154bd4db5a', sequence_id=248, task_id='0927b469-5aed-4804-aa53-79a6af70f76f', content=TextContent(author='user', content='Hello what can you do? EVENT NUM: 1', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 8, 14, 5, 54, 53, 141757, tzinfo=TzInfo(UTC)))
Event(id='797aca62-6260-4c4d-a89b-43e33ecfdc30', agent_id='c9fc2e91-df7a-42b4-bc79-fe154bd4db5a', sequence_id=249, task_id='0927b469-5aed-4804-aa53-79a6af70f76f', content=TextContent(author='user', content='Hello what 

In [42]:

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "data", "author": "user", "data": {"clear_queue": True, "cancel_running_tasks": True}},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='e4204e25-dba1-428a-a278-e5bf16464cbc', agent_id='c9fc2e91-df7a-42b4-bc79-fe154bd4db5a', sequence_id=277, task_id='0927b469-5aed-4804-aa53-79a6af70f76f', content=DataContent(author='user', data={'clear_queue': True, 'cancel_running_tasks': True}, style='static', type='data'), created_at=datetime.datetime(2025, 8, 14, 5, 55, 23, 716517, tzinfo=TzInfo(UTC)))


In [30]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=5,
)

╭──────────────────────── AGENT [08/14/2025 05:49:07] ─────────────────────────╮
│ I just cleared the queue of events that were received. Total cleared events: │
│ 1                                                                            │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 5 seconds - returning collected messages
